In [1]:
import utils
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload

# Create grid of 3D delays
Create a grid of delays for points spaced 2 meters apart from each other on a 3D 4-recorder grid (square in X and Y dimensions)

In [3]:
### Dataframe of recorders on plane
r_locs_3d = pd.DataFrame(
    {
        'r1': (0, 0, 2),
        'r2':(0, 25, 0),
        'r3':(25, 0, 1),
        'r4':(25, 25, 0)
    },
    index = ['x', 'y', 'z']).T

### Heights for each set of delays
heights = np.array([0, 10])

### Create grid of delays
temp = 20.0
(s_locs, true_delays) = utils.make_delay_grid(
    r_locs_3d, heights, spacing = 2, temperature = temp)

In [4]:
s_locs.head()

,x,y,z
0,1.0,1.0,0.0
1,1.0,1.0,10.0
2,3.0,1.0,0.0
3,3.0,1.0,10.0
4,5.0,1.0,0.0


In [5]:
true_delays.head()

,r1,r2,r3,r4
0,0.007137,0.069988,0.070048,0.098892
1,0.023670,0.075810,0.074739,0.103095
2,0.010902,0.070471,0.064232,0.094861
3,0.025064,0.076257,0.069317,0.099235
4,0.015959,0.071428,0.058418,0.091025


# Add error to delays
For each delay, run 5 different simulations of delay calculation error by adding "jitter" to delays drawn from a uniform distribution

In [6]:
### Error distributed Uniform(-0.2, 0.2)
ms_error = 0.2

### Number of trials to run
num_trials = 5

### Add the error
ms_error = 0.20
s_error = ms_error/100
jittered = utils.make_jitter_trials(
    df = true_delays,
    amt = s_error,
    trials = num_trials,
)

In [7]:
jittered[0].head()

,r1,r2,r3,r4
0,0.0,0.062177,0.062283,0.089155
1,0.0,0.051466,0.050440,0.076824
2,0.0,0.058895,0.052702,0.081359
3,0.0,0.050519,0.043625,0.071571
4,0.0,0.054796,0.041831,0.072466


# Localize the 3D sounds

Try localizing one of the jitter settings

In [8]:
%autoreload
import pysoundfinder as pysf
localized = []
for sound_num in jittered[0].index:
    localized.append(pysf.localize_sound(
        positions = r_locs_3d,
        times = jittered[0].iloc[sound_num],
        temp = 20.0,
        invert_alg = 'gps', #options: 'lstsq', 'gps'
        center = True, #True = original Sound Finder behavior
        pseudo = True #False = original Sound Finder
    ))
results = pd.DataFrame(np.array(localized).reshape(jittered[0].shape))
results.head()

/Users/tessa/Code/loca9/pysoundfinder/pysoundfinder.py:138: UserWarning: centering
  warnings.warn("centering")
/Users/tessa/Code/loca9/pysoundfinder/pysoundfinder.py:226: UserWarning: Discriminant negative--set to zero. Solution may be inaccurate. Inspect final value of output array
  warnings.warn("Discriminant negative--set to zero. Solution may be inaccurate. Inspect final value of output array", UserWarning)


,0,1,2,3
0,2.495695,2.593919,2.137477,-1.041774
1,3.202065,2.874966,0.129752,-4.692172
2,4.399310,2.657545,2.337009,-2.198460
3,4.927240,2.996906,2.890086,-5.250757
4,6.308473,2.787865,2.658659,-3.683586


Use the utility to do the same thing

In [9]:
%autoreload
import pysoundfinder
results = utils.localize_pysf(
    rel_sounds = jittered[0],
    r_locs = r_locs_3d,
    invert_alg = 'lstsq')
results.head()

,x,y,z,r
0,2.495695,2.593919,2.137477,-1.041774
1,3.202065,2.874966,0.129752,-4.692172
2,4.399310,2.657545,2.337009,-2.198460
3,4.927240,2.996906,2.890086,-5.250757
4,6.308473,2.787865,2.658659,-3.683586


# Localize the 3D sounds in 2D

In [10]:
### Take recorder locations to 2D
r_locs = r_locs_3d[['x', 'y']]

Run trials on all settings, with all levels of jitter.

In [12]:
%autoreload
import utils
### Localize using PYSF
py_est_dict = {}
# All combinations
for c in [True, False]:
    for s in [True, False]:
        for a in ['gps', 'lstsq']:
            key = f'ALGO: {a}, CENTER: {c}, SOS: {s}'
            py_est = [None] * num_trials

            # Localize each trial in the jitter array
            for idx, jitter in enumerate(jittered):
                py_trial = utils.localize_pysf(
                    rel_sounds = jittered[idx],
                    r_locs = r_locs,
                    invert_alg = a,
                    center_array = c,
                    use_sos_selection = s,
                    temperature = temp
                )

                py_est[idx] = py_trial
                exit()

            py_est_dict[key] = py_est


keys = list(py_est_dict.keys())

/Users/tessa/Code/loca9/pysoundfinder/pysoundfinder.py:150: UserWarning: not centering
  warnings.warn("not centering")


# Plot error

In [ ]:
### Find error
error_types = utils.calc_all_errors(
    original_locs = s_locs,
    dfs = py_est_dict)

### Average error across trials
avg_err_dict = {key: [None] for key in keys}
for key in keys:
    avg_err_dict[key] = utils.avg_error(error_types[key])
    
### Create plot
utils.make_err_plot(
    err_dfs = avg_err_dict,
    recs = r_locs.T,
    ms_error = ms_error,
    trials = num_trials,
#    vert_max = ,
    sep_by = [0, 10],
    std_dev = False,
    fig_size = (10, 30),
    h_space = 1
).show()